In [1]:
# %reload_ext autoreload
# %matplotlib notebook
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns


import math




In [2]:
import statsmodels.api as sm

In [3]:
# from sklearn.datasets import load_svmlight_file


# def read_gas_array_data(dir_path):
#     Xall = []
#     Yall = []
#     for i in xrange(1, 10):
#         Xb, Yb = load_svmlight_file(dir_path + "batch%d.dat" % i)
#         Xall.append(np.array(Xb.todense()))
#         Yall.append(np.array(Yb))
#     return np.concatenate(tuple(Xall)), np.concatenate(tuple(Yall))

# Xall, Yall = read_gas_array_data("uci-data/Gas_Sensor_Array_Drift_Dataset_Data_Set/")

In [4]:

# pd.DataFrame(Yall).describe()


In [5]:
# plt.figure()
# plt.hist(Yall)


In [6]:
# print Xall.shape, Yall.shape

In [7]:
# sm_mnlr = sm.MNLogit(Yall, sm.add_constant(Xall, prepend=False))
# sm_mnlr_fit = sm_mnlr.fit(method="bfgs")

In [8]:
def read_musk_data(file_path):
    with open(file_path, 'r') as fin:
        dat_all = []
        for line in fin:
            line = line.strip()[:-1].split(",")[2:]
            dat_all.append(line)
    dat_all = np.array(dat_all, dtype=np.float)
    return dat_all[:, :-1], dat_all[:, -1]

            
Xm, Ym = read_musk_data("uci-data/musk_clean2.data")

In [9]:
print Xm.shape
print Ym.shape

(6598, 166)
(6598,)


In [10]:
n, p = Xm.shape

In [11]:
pd.DataFrame(Ym).describe()

,0
count,6598.000000
mean,0.154138
std,0.361108
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [12]:
from sklearn import preprocessing

In [13]:
scaler = preprocessing.StandardScaler().fit(Xm)

In [14]:
sm_lr = sm.Logit(Ym, sm.add_constant(scaler.transform(Xm), prepend=False))
sm_lr_fit = sm_lr.fit()

Optimization terminated successfully.
         Current function value: 0.109768
         Iterations 12


In [15]:
print sm_lr_fit.cov_params()

[[  1.38115146e+00  -3.50590412e-02  -1.82661806e-01 ...,   8.50703782e-02
   -8.34996610e-02   9.42777144e-02]
 [ -3.50590412e-02   2.84149520e-01  -6.77511087e-02 ...,  -6.85519595e-04
    1.03233680e-02  -2.08633977e-02]
 [ -1.82661806e-01  -6.77511087e-02   2.37082472e+00 ...,   4.92605324e-02
    3.06908919e-02   5.12668999e-02]
 ..., 
 [  8.50703782e-02  -6.85519595e-04   4.92605324e-02 ...,   4.36333129e-01
   -5.40338036e-02   4.57211553e-02]
 [ -8.34996610e-02   1.03233680e-02   3.06908919e-02 ...,  -5.40338036e-02
    1.09863270e-01  -5.86971805e-03]
 [  9.42777144e-02  -2.08633977e-02   5.12668999e-02 ...,   4.57211553e-02
   -5.86971805e-03   6.02271206e-02]]


In [16]:

plt.figure()
plt.imshow(sm_lr_fit.cov_params(), cmap='viridis')
plt.colorbar()

In [17]:
plt.figure()
plt.plot(sm_lr_fit.cov_params().diagonal())

In [18]:

# NBS = 15000

# w_bs = []

# for _ in xrange(NBS):
#     ind_bs = np.random.choice(n, size=n, replace=True)
#     bs_lr = sm.Logit(Ym[ind_bs], sm.add_constant(scaler.transform(Xm[ind_bs, :]), prepend=False))
#     bs_lr_fit = bs_lr.fit(disp=False)
#     w_bs.append(bs_lr_fit.params.ravel().tolist())
    

# w_bs = np.array(w_bs)


import cPickle as pickle

# pickle.dump(w_bs, open("PICKLE_uci_musk_BS", 'wb'))


w_bs = pickle.load(open("/u/lty/scratch/data/PICKLE_uci_musk_BS", 'rb'))


cov_bs = np.cov(w_bs.T)

In [19]:
plt.figure()
plt.plot(sm_lr_fit.cov_params().diagonal(), 'b', label="inv Fisher")
plt.plot(cov_bs.diagonal(), 'r', label="bootstrap")




In [20]:

from __future__ import division


from __future__ import division
import tensorflow as tf



import statsmodels.api as sm



def sgd_avg_infer(X_, Y_, lr, 
                  size_batch, 
                  burn_in, n_avg, n_skip, n_sample):
    lr = float(lr)
    
    n, p = X_.shape
    
    print n, p

    sm_lr = sm.Logit(Y_, X_)
    sm_lr_fit = sm_lr.fit(disp=False)
    
    
    X = tf.placeholder(tf.float64, [None, p])
    
    Y = tf.placeholder(tf.float64, [None, 1])
    
    w = tf.Variable(initial_value=sm_lr_fit.params.reshape((-1, 1)), dtype=tf.float64)

    print(w)
    
    
    log_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.matmul(X, w), labels=Y))
    
    opt = tf.train.GradientDescentOptimizer(lr).minimize(log_loss)
    
    init = tf.global_variables_initializer()
    
    
    
    n_iter = burn_in + (n_avg + n_skip) * n_sample + 3
    
    
    w_avg_samples = []
    
    w_all = []
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i_ in xrange(n_iter):
            ind_batch = np.random.choice(n, size=size_batch, replace=True)
            
            Xb = X_[ind_batch, :].reshape((-1, p))
            Yb = Y_[ind_batch].reshape((-1, 1))
            
            sess.run([opt], feed_dict={X: Xb, Y: Yb})
            
            if i_ >= burn_in:
                if (i_ - burn_in) % (n_avg + n_skip) < n_avg:
                    wb = w.eval(session=sess).ravel().tolist()
                    w_avg_samples.append(wb)
                elif (i_ - burn_in) % (n_avg + n_skip) == n_avg:
                    w_avg_samples = np.array(w_avg_samples)
                    w_avg = np.mean(w_avg_samples, axis=0)
                    w_all.append(w_avg.ravel().tolist())
                    w_avg_samples = []
    
    return np.array(w_all)  





In [21]:
import multiprocessing


In [22]:
burn_in = 20

# n_avg = 5000
# n_skip = 100
# n_sample = 5000


n_avg = 2000
n_skip = 70

n_sample = 1000

lr = .05

size_batch = 10





mpool = multiprocessing.Pool(processes=8)





mpool_args = []


#########

for n_avg in [7000, 14000, 21000, 28000, 35000]:
    for lr in [1, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001]:
        for size_batch in [2, 4, 8, 16]:

#             n_avg = 2000
            n_skip = 1000

            n_sample = 5000

#             lr = .05

#             size_batch = 10

            mpool_args.append((sm.add_constant(Xm, prepend=False), Ym, lr, size_batch, burn_in, n_avg, n_skip, n_sample))


#########


    
# w_sgd_avg = sgd_avg_infer(sm.add_constant(Xm, prepend=False), Ym, lr, size_batch, burn_in, n_avg, n_skip, n_sample)
    

results = [mpool.apply_async(sgd_avg_infer, mpool_arg) for mpool_arg in mpool_args]
    
    
mpool.close()
mpool.join()



import cPickle as pickle

# pickle.dump(sim_res, open("logistic-large_2017_03_01", 'wb'))

results_ = [r.get() for r in results]

pickle.dump({"mpool_args": mpool_args, "results": results_}, open("/u/lty/scratch/results/check-data-musk_parallel_03_21_17_21", 'wb'))


6598 167
6598 167
6598 167
Tensor("Variable/read:0", shape=(167, 1), dtype=float64)
Tensor("Variable/read:0", shape=(167, 1), dtype=float64)
Tensor("Variable/read:0", shape=(167, 1), dtype=float64)


In [23]:

# plt.figure()#figsize=(12,8))

# line_sgd, = plt.plot(np.mean(w_sgd_avg, axis=0), label='SGD Inference')
# line_fisher, = plt.plot(sm_lr_fit.params, label = 'Fisher')


# plt.legend(handles=[line_sgd, line_fisher])

# plt.title('Optimization output')
# plt.show()

NameError: name 'w_sgd_avg' is not defined

In [ ]:
# cov_estimate = np.cov(w_sgd_avg.T) * size_batch / n * n_avg 

# from sklearn.covariance import EmpiricalCovariance, MinCovDet


# # cov_estimate2 = MinCovDet().fit(w_sgd_avg) 
# # cov_estimate2 = size_batch / n * n_avg * cov_estimate2.covariance_


# plt.figure()
# #plt.figure(figsize=(12,8))


# line_sgd, = plt.plot(cov_estimate.diagonal(), 'r', label='SGD')

# #line_fisher, = plt.plot(sm_lr_fit.cov_params().diagonal(), label = 'Fisher')

# line_bs, = plt.plot(np.cov(w_bs.T).diagonal(), 'b', label="bootstrap")

# # plt.plot(sm_lr_fit.cov_params().diagonal(), 'b', label="inverse Fisher")


# plt.legend()#handles=[line_sgd, line_fisher, line_bs])

# #plt.title('Diagonal')

# # plt.ylim((0, 0.4))

# plt.legend(prop={'size': 23})

# plt.show()

# # plt.savefig("logistic-11d-rho-0_6--diag.pdf", format='pdf')